# Plan to WIN A MEDAL – Cdiscount Image Classification

Objectives:
- Build a robust, GPU-accelerated image classification pipeline for 5k+ classes
- Establish trustworthy CV and fast baselines; iterate to strong accuracy
- Produce a valid submission.csv ASAP, then improve

Workflow:
1) Environment & GPU
- Verify GPU availability (nvidia-smi), install PyTorch CUDA 12.1 stack if needed
- Create constraints to lock torch versions; avoid drift

2) Data understanding
- Inspect files: train.bson, test.bson, train_example.bson, category_names.csv, sample_submission.csv
- Determine schema: BSON contains products, each with multiple images and category_id (train only)
- Map category_id <-> class index; confirm number of classes

3) Reader & Dataset
- Implement BSON streaming reader (avoid loading entire file in memory)
- For train: expand to (product_id, image_index) samples with label; for test: (product_id, image_index)
- Use JPEG decode from bytes (PIL) and standard transforms; cache occasional decoded bytes if RAM allows

4) CV Protocol
- StratifiedGroupKFold by product_id grouped, stratified on category_id
- Single deterministic split first (e.g., 90/10) to iterate fast; later move to 5-fold if time
- Aggregate per-product predictions by mean of image logits

5) Baseline Model (fast)
- Use torchvision/timm pretrained backbone at 180x180: e.g., efficientnet_b0, convnext_tiny, or resnet50
- Replace head with num_classes; use label-smoothing CE
- Optim: AdamW, cosine schedule, warmup; amp autocast; EMA head optional
- Data aug: RandomResizedCrop(180), HFlip; test-time: center-crop only

6) Training Loop
- Mixed precision, gradient accumulation if needed; log every N steps with elapsed time
- Early stopping by val accuracy; save best weights per fold

7) Inference
- Predict per-image, aggregate per-product (mean logits), argmax category_id
- Write submission.csv with correct columns

8) Iterations to Medal
- Resolution sweep: 224, 256 short runs
- Stronger backbones: convnext_tiny/base, efficientnet_b3, vit_tiny/base (if time)
- Class-balanced sampling / reweighting; focal loss trial
- Ensembling: average logits from 2–3 diverse seeds/backbones

9) Validation Rigor
- Save folds, OOF logits; compute accuracy by product and by image
- Inspect per-class accuracy and top-k errors; adjust sampling/augmentations

10) Time Management
- Start with train_example.bson to validate pipeline and submission
- Scale to a subset of train (e.g., 200k products) for fast baseline
- Only then run full training; keep logs; consider interrupting long runs based on expert advice

In [ ]:
# Install BSON dependency (provided by pymongo)
import sys, subprocess
def pip(*args):
    print('> pip', *args, flush=True)
    return subprocess.run([sys.executable, '-m', 'pip', *args], check=True)

pip('install', 'pymongo')
print('pymongo installed for bson import')

In [ ]:
# Environment & GPU check + Torch cu121 install, quick data peek
import os, sys, shutil, subprocess, time, json, bson, struct, io
from pathlib import Path
import pandas as pd

def run(cmd):
    print('>',' '.join(cmd), flush=True)
    return subprocess.run(cmd, check=False, text=True, capture_output=True)

print('=== NVIDIA SMI ===', flush=True)
print(run(['bash','-lc','nvidia-smi || true']).stdout)

# Clean any preinstalled torch stacks to avoid CUDA mismatch
for pkg in ("torch","torchvision","torchaudio"):
    subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", pkg], check=False)

# Remove possible shadow dirs
for d in (
    "/app/.pip-target/torch",
    "/app/.pip-target/torchvision",
    "/app/.pip-target/torchaudio",
    "/app/.pip-target/torch-2.4.1.dist-info",
    "/app/.pip-target/torchvision-0.19.1.dist-info",
    "/app/.pip-target/torchaudio-2.4.1.dist-info",
    "/app/.pip-target/torchgen",
    "/app/.pip-target/functorch",
):
    if os.path.exists(d):
        print('Removing', d, flush=True)
        shutil.rmtree(d, ignore_errors=True)

def pip(*args):
    print('> pip', *args, flush=True)
    return subprocess.run([sys.executable, '-m', 'pip', *args], check=True)

# Install exact cu121 torch stack
pip('install',
    '--index-url','https://download.pytorch.org/whl/cu121',
    '--extra-index-url','https://pypi.org/simple',
    'torch==2.4.1','torchvision==0.19.1','torchaudio==2.4.1')

# Freeze torch versions for later installs
Path('constraints.txt').write_text('torch==2.4.1\ntorchvision==0.19.1\ntorchaudio==2.4.1\n')

import torch
print('torch:', torch.__version__, 'CUDA build:', getattr(torch.version,'cuda',None))
print('CUDA available:', torch.cuda.is_available())
if torch.cuda.is_available():
    print('GPU:', torch.cuda.get_device_name(0))
assert str(getattr(torch.version,'cuda','')).startswith('12.1'), f"Wrong CUDA build: {torch.version.cuda}"
assert torch.cuda.is_available(), 'CUDA not available'

print('\n=== Repo contents ===')
for p in sorted(Path('.').glob('*')):
    try:
        sz = p.stat().st_size
    except Exception:
        sz = -1
    print(f"{p.name}\t{sz/1e6:.1f} MB")

# Quick CSV peeks
if Path('sample_submission.csv').exists():
    ss = pd.read_csv('sample_submission.csv')
    print('\nSample submission shape:', ss.shape)
    print(ss.head())
if Path('category_names.csv').exists():
    cat = pd.read_csv('category_names.csv')
    print('\nCategory names shape:', cat.shape)
    print(cat.head())

# Peek a few BSON docs from train_example to confirm schema
def peek_bson(path, n=3):
    print(f'\nPeeking {n} docs from {path}...')
    c = 0
    with open(path, 'rb') as f:
        while c < n:
            len_bytes = f.read(4)
            if not len_bytes or len(len_bytes) < 4:
                break
            (doc_len,) = struct.unpack('<i', len_bytes)
            rest = f.read(doc_len - 4)
            if len(rest) < doc_len - 4:
                break
            try:
                d = bson.BSON(len_bytes + rest).decode()
            except Exception as e:
                print('Decode error:', e)
                continue
            keys = list(d.keys())
            print('Doc keys:', keys)
            print({k: type(d[k]).__name__ for k in keys})
            if '_id' in d:
                print('product_id:', d['_id'])
            if 'category_id' in d:
                print('category_id:', d['category_id'])
            if 'imgs' in d:
                print('n_imgs:', len(d['imgs']))
            c += 1

if Path('train_example.bson').exists():
    peek_bson('train_example.bson', n=3)
elif Path('train.bson').exists():
    # Fallback to full train if example is absent (will be slow, we only read 1-3 docs)
    peek_bson('train.bson', n=3)

print('\nSETUP COMPLETE', flush=True)

In [ ]:
# BSON indexing utilities: build product-level index and category mappings
import os, struct, json, io
from pathlib import Path
import pandas as pd

def build_bson_index(bson_path, out_index_csv, is_train=True, max_docs=None, log_every=100000):
    rows = []
    bson_path = Path(bson_path)
    total = bson_path.stat().st_size
    with open(bson_path, 'rb') as f:
        ofs = 0
        i = 0
        while True:
            len_bytes = f.read(4)
            if not len_bytes or len(len_bytes) < 4:
                break
            (doc_len,) = struct.unpack('<i', len_bytes)
            rest = f.read(doc_len - 4)
            if len(rest) < doc_len - 4:
                break
            try:
                import bson as _bson
                d = _bson.BSON(len_bytes + rest).decode()
            except Exception:
                # skip corrupted doc; advance offset anyway
                ofs += doc_len
                i += 1
                continue
            prod_id = d.get('_id')
            imgs = d.get('imgs', [])
            n_imgs = len(imgs) if isinstance(imgs, list) else 0
            cat_id = d.get('category_id') if is_train else None
            rows.append({
                'offset': ofs,
                '_id': prod_id,
                'n_imgs': n_imgs,
                'category_id': cat_id if is_train else pd.NA,
                'doc_len': doc_len,
            })
            ofs += doc_len
            i += 1
            if log_every and (i % log_every == 0):
                print(f"Indexed {i:,} docs, pos {ofs/1e9:.3f} GB / {total/1e9:.3f} GB", flush=True)
            if max_docs is not None and i >= max_docs:
                break
    df = pd.DataFrame(rows)
    df.to_csv(out_index_csv, index=False)
    print(f"Saved index: {out_index_csv} with {len(df):,} rows", flush=True)
    return df

def build_category_mapping_from_index(train_index_csv, out_map_json):
    df = pd.read_csv(train_index_csv)
    cats = sorted(df['category_id'].dropna().astype(int).unique().tolist())
    cat2idx = {int(c): i for i, c in enumerate(cats)}
    idx2cat = {i: int(c) for i, c in enumerate(cats)}
    payload = {'cat2idx': cat2idx, 'idx2cat': idx2cat, 'num_classes': len(cats)}
    Path(out_map_json).write_text(json.dumps(payload))
    print(f"Saved category mapping to {out_map_json} (num_classes={len(cats)})", flush=True)
    return payload

# Dry-run helpers (won't execute automatically):
print('Indexing utilities ready. Example usage:', flush=True)
print("- build_bson_index('train_example.bson', 'train_example_index.csv', is_train=True)", flush=True)
print("- build_bson_index('test.bson', 'test_index.csv', is_train=False, max_docs=50000)  # for smoke", flush=True)
print("- build_bson_index('train.bson', 'train_index.csv', is_train=True, log_every=200000)", flush=True)
print("- build_category_mapping_from_index('train_index.csv', 'category_mapping.json')", flush=True)

In [ ]:
# Build tiny indices/mapping on train_example to validate pipeline
from pathlib import Path
import pandas as pd, json

if Path('train_example.bson').exists():
    df_idx = build_bson_index('train_example.bson', 'train_example_index.csv', is_train=True)
    print(df_idx.head())
    mapping = build_category_mapping_from_index('train_example_index.csv', 'category_mapping_example.json')
    print('Mapping keys:', list(mapping.keys()), 'num_classes:', mapping['num_classes'])
else:
    print('train_example.bson not found; skipping example index build')

In [ ]:
# Build full train/test indices with explicit dtypes and save as parquet (optionally smoke-limit)
import pandas as pd, pyarrow as pa, pyarrow.parquet as pq, struct, os, json, time
from pathlib import Path
import bson as _bson

def build_index_parquet(bson_path: str, out_parquet: str, is_train: bool, max_docs=None, log_every=200000):
    t0 = time.time()
    rows = []
    p = Path(bson_path)
    total = p.stat().st_size
    with open(p, 'rb') as f:
        ofs = 0
        i = 0
        while True:
            len_bytes = f.read(4)
            if not len_bytes or len(len_bytes) < 4:
                break
            (doc_len,) = struct.unpack('<i', len_bytes)
            rest = f.read(doc_len - 4)
            if len(rest) < doc_len - 4:
                break
            try:
                d = _bson.BSON(len_bytes + rest).decode()
            except Exception:
                ofs += doc_len; i += 1
                continue
            prod_id = int(d.get('_id'))
            imgs = d.get('imgs', [])
            n_imgs = len(imgs) if isinstance(imgs, list) else 0
            if n_imgs <= 0:
                ofs += doc_len; i += 1
                continue
            cat_id = int(d['category_id']) if is_train else None
            rows.append((ofs, prod_id, n_imgs, cat_id if is_train else None, doc_len))
            ofs += doc_len
            i += 1
            if log_every and (i % log_every == 0):
                print(f"Indexed {i:,} docs, pos {ofs/1e9:.3f} GB / {total/1e9:.3f} GB, elapsed {time.time()-t0:.1f}s", flush=True)
            if max_docs is not None and i >= max_docs:
                break
    if not rows:
        print('No rows parsed; nothing to write.')
        return None
    # Build DataFrame with explicit dtypes
    df = pd.DataFrame(rows, columns=['offset','_id','n_imgs','category_id','doc_len'])
    dtypes = {
        'offset': 'int64',
        '_id': 'int64',
        'n_imgs': 'int16',
        'doc_len': 'int32'
    }
    if is_train:
        dtypes['category_id'] = 'Int64'
    else:
        dtypes['category_id'] = 'Int64'
        df['category_id'] = pd.NA
    df = df.astype(dtypes)
    table = pa.Table.from_pandas(df, preserve_index=False)
    pq.write_table(table, out_parquet, compression='zstd')
    print(f"Saved {len(df):,} rows to {out_parquet} in {time.time()-t0:.1f}s", flush=True)
    return out_parquet

def ensure_full_indices(smoke=False):
    # If smoke=True, limit docs for quick run
    train_out = 'train_index.parquet'
    test_out = 'test_index.parquet'
    if not Path(train_out).exists():
        build_index_parquet('train.bson', train_out, is_train=True, max_docs=(50000 if smoke else None))
    else:
        print(f'{train_out} exists, skipping.')
    if not Path(test_out).exists():
        build_index_parquet('test.bson', test_out, is_train=False, max_docs=(50000 if smoke else None))
    else:
        print(f'{test_out} exists, skipping.')
    # Build mapping if train index created
    if Path(train_out).exists():
        df_tr = pd.read_parquet(train_out, columns=['category_id'])
        cats = sorted(df_tr['category_id'].dropna().astype(int).unique().tolist())
        cat2idx = {int(c): i for i, c in enumerate(cats)}
        idx2cat = {i: int(c) for i, c in enumerate(cats)}
        payload = {'cat2idx': cat2idx, 'idx2cat': idx2cat, 'num_classes': len(cats)}
        Path('category_mapping.json').write_text(json.dumps(payload))
        print(f"Saved category_mapping.json (num_classes={len(cats)})", flush=True)

print('Full-index builder ready. Call ensure_full_indices(smoke=True) for a quick pass, or smoke=False for full run.', flush=True)

In [ ]:
# Smoke-build indices (50k docs) for train/test to validate performance before full pass
import time
t0 = time.time()
print('Starting ensure_full_indices(smoke=True)...')
ensure_full_indices(smoke=True)
print(f'Done smoke indexing in {time.time()-t0:.1f}s')

In [3]:
# Dataset and DataLoader utilities for BSON with per-worker file handles
import os, io, random, struct
from pathlib import Path
import pandas as pd
from PIL import Image, ImageFile
import torch
from torch.utils.data import Dataset, DataLoader, Subset
import torchvision.transforms as T
import bson as _bson

ImageFile.LOAD_TRUNCATED_IMAGES = True

def load_mapping(path='category_mapping.json'):
    import json
    m = json.loads(Path(path).read_text())
    cat2idx = {int(k): int(v) for k,v in m['cat2idx'].items()}
    idx2cat = {int(k): int(v) for k,v in m['idx2cat'].items()}
    num_classes = int(m['num_classes'])
    return cat2idx, idx2cat, num_classes

def read_bson_doc_at(fh, offset):
    # fh is a file handle opened in 'rb'
    fh.seek(offset)
    len_bytes = fh.read(4)
    if not len_bytes or len(len_bytes) < 4:
        return None
    (doc_len,) = struct.unpack('<i', len_bytes)
    rest = fh.read(doc_len - 4)
    if len(rest) < doc_len - 4:
        return None
    try:
        d = _bson.BSON(len_bytes + rest).decode()
        return d
    except Exception:
        return None

class BSONProductTrain(Dataset):
    def __init__(self, index_df: pd.DataFrame, bson_path: str, cat2idx: dict, image_size=224):
        self.df = index_df.reset_index(drop=True)
        self.bson_path = str(bson_path)
        self.cat2idx = cat2idx
        self._fh = None  # per-worker handle, opened lazily
        # Strong but stable aug pipeline
        self.transform = T.Compose([
            T.RandomResizedCrop(image_size, scale=(0.5, 1.0), ratio=(0.75, 1.33)),
            T.RandomHorizontalFlip(0.5),
            T.ColorJitter(0.2, 0.2, 0.2, 0.05),
            T.ToTensor(),
            T.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
            T.RandomErasing(p=0.18, scale=(0.02, 0.2), ratio=(0.3, 3.3)),
        ])

    def __len__(self):
        return len(self.df)

    def _ensure_fh(self):
        if self._fh is None:
            self._fh = open(self.bson_path, 'rb', buffering=0)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        self._ensure_fh()
        d = read_bson_doc_at(self._fh, int(row['offset']))
        # Always set label from index row's category_id mapping (do NOT force class 0 on failures)
        cat_id_row = int(row['category_id']) if pd.notna(row['category_id']) else None
        y = self.cat2idx.get(cat_id_row, 0) if cat_id_row is not None else 0
        if d is None:
            # return a dummy image but keep correct label
            img = Image.new('RGB', (256,256), color=(128,128,128))
            return self.transform(img), y
        imgs = d.get('imgs', [])
        # pick one random image
        choice = random.randrange(len(imgs)) if imgs else 0
        pic_bytes = imgs[choice]['picture'] if imgs else None
        try:
            img = Image.open(io.BytesIO(pic_bytes)).convert('RGB') if pic_bytes is not None else Image.new('RGB',(256,256))
        except Exception:
            img = Image.new('RGB',(256,256), color=(128,128,128))
        x = self.transform(img)
        return x, y

class BSONImageEval(Dataset):
    # Yields one image per item for val/test; aggregate per-product outside
    def __init__(self, index_df: pd.DataFrame, bson_path: str, include_label: bool, cat2idx: dict | None, image_size=224):
        # expand to per-image rows
        rows = []
        for _, r in index_df.iterrows():
            for k in range(int(r['n_imgs'])):
                rows.append((int(r['offset']), int(r['_id']), k, int(r['category_id']) if include_label and pd.notna(r['category_id']) else None))
        self.df = pd.DataFrame(rows, columns=['offset','_id','img_idx','category_id'])
        # Cheap IO win: process in offset order to improve locality
        self.df.sort_values('offset', inplace=True)
        self.bson_path = str(bson_path)
        self.include_label = include_label
        self.cat2idx = cat2idx
        self._fh = None
        self.transform = T.Compose([
            T.Resize(256),
            T.CenterCrop(image_size),
            T.ToTensor(),
            T.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
        ])

    def __len__(self):
        return len(self.df)

    def _ensure_fh(self):
        if self._fh is None:
            self._fh = open(self.bson_path, 'rb', buffering=0)

    def __getitem__(self, idx):
        r = self.df.iloc[idx]
        self._ensure_fh()
        d = read_bson_doc_at(self._fh, int(r['offset']))
        imgs = d.get('imgs', []) if d is not None else []
        pic_bytes = imgs[int(r['img_idx'])]['picture'] if d is not None and imgs else None
        try:
            img = Image.open(io.BytesIO(pic_bytes)).convert('RGB') if pic_bytes is not None else Image.new('RGB',(256,256))
        except Exception:
            img = Image.new('RGB',(256,256), color=(128,128,128))
        x = self.transform(img)
        if self.include_label:
            cat_id = int(r['category_id']) if r['category_id'] is not None and pd.notna(r['category_id']) else None
            y = self.cat2idx.get(cat_id, 0) if (self.cat2idx is not None and cat_id is not None) else -1
            return x, int(r['_id']), int(r['img_idx']), y
        else:
            return x, int(r['_id']), int(r['img_idx'])

def make_dataloaders_for_smoke(train_index_path='train_index.parquet', batch_size=64, num_workers=8):
    cat2idx, idx2cat, num_classes = load_mapping('category_mapping.json')
    dft = pd.read_parquet(train_index_path)
    # small subset for smoke: 2k products
    dft = dft.sample(n=min(2000, len(dft)), random_state=42).reset_index(drop=True)
    # split 90/10 stratified by category at product level
    from sklearn.model_selection import StratifiedGroupKFold
    y = dft['category_id'].astype(int)
    g = dft['_id'].astype(int)
    skf = StratifiedGroupKFold(n_splits=10, shuffle=True, random_state=42)
    tr_idx, va_idx = next(skf.split(dft, y, g))
    df_tr = dft.iloc[tr_idx].reset_index(drop=True)
    df_va = dft.iloc[va_idx].reset_index(drop=True)
    ds_tr = BSONProductTrain(df_tr, 'train.bson', cat2idx, image_size=192)
    ds_va = BSONImageEval(df_va, 'train.bson', include_label=True, cat2idx=cat2idx, image_size=192)
    dl_tr = DataLoader(ds_tr, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True, persistent_workers=True, prefetch_factor=4, drop_last=True)
    dl_va = DataLoader(ds_va, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True, persistent_workers=True, prefetch_factor=4)
    return dl_tr, dl_va, num_classes, (df_tr, df_va)

print('Dataset classes ready. Next: create loaders with make_dataloaders_for_smoke() and wire a minimal ConvNeXt-Tiny training loop.', flush=True)

Dataset classes ready. Next: create loaders with make_dataloaders_for_smoke() and wire a minimal ConvNeXt-Tiny training loop.


In [ ]:
# Install timm, create smoke loaders, and sanity-check a couple batches
import sys, subprocess, time, torch
from pathlib import Path

def pip(*args):
    print('> pip', *args, flush=True)
    return subprocess.run([sys.executable, '-m', 'pip', *args], check=True)

# Install timm honoring torch constraints but without touching torch stack
if not Path('installed_timm.flag').exists():
    pip('install', '-c', 'constraints.txt', 'timm==1.0.9', '--upgrade-strategy', 'only-if-needed', '--no-deps')
    Path('installed_timm.flag').write_text('ok')
    print('timm installed')
else:
    print('timm already installed')

torch.backends.cudnn.benchmark = True

dl_tr, dl_va, num_classes, (df_tr, df_va) = make_dataloaders_for_smoke(batch_size=128, num_workers=12)
print('num_classes:', num_classes, 'train products:', len(df_tr), 'val products:', len(df_va))

# Fetch a couple of train batches and move to GPU to validate shapes and throughput
t0 = time.time()
for bi, (x, y) in enumerate(dl_tr):
    x = x.cuda(non_blocking=True)
    y = y.cuda(non_blocking=True)
    if bi == 0:
        print('Train batch 0:', x.shape, y.shape, 'GPU mem (MB):', torch.cuda.memory_allocated()/1e6)
    if bi >= 2:
        break
print('Train fetch elapsed:', time.time()-t0, 's')

# Fetch a couple of val batches
t1 = time.time()
for bi, batch in enumerate(dl_va):
    # dl_va yields (x, _id, img_idx, y)
    x = batch[0].cuda(non_blocking=True)
    if bi == 0:
        print('Val batch 0 x-shape:', x.shape)
    if bi >= 2:
        break
print('Val fetch elapsed:', time.time()-t1, 's')

print('Smoke loaders OK')

In [ ]:
# Minimal ConvNeXt-Tiny training loop (smoke) with product-level validation
import time, math, collections
import torch
import torch.nn as nn
import torch.optim as optim
from torch.cuda.amp import autocast, GradScaler
import timm

def train_one_epoch(model, loader, optimizer, scaler, device, log_every=20):
    model.train()
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1).to(device)
    total, correct, running_loss = 0, 0, 0.0
    t0 = time.time()
    for i, (x, y) in enumerate(loader):
        x = x.to(device, non_blocking=True)
        y = y.to(device, non_blocking=True)
        optimizer.zero_grad(set_to_none=True)
        with autocast():
            logits = model(x)
            loss = criterion(logits, y)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        running_loss += loss.item() * x.size(0)
        preds = logits.argmax(dim=1)
        total += y.size(0)
        correct += (preds == y).sum().item()
        if (i+1) % log_every == 0:
            elapsed = time.time() - t0
            print(f"  iter {i+1}/{len(loader)}  loss={(running_loss/total):.4f}  acc={(correct/total):.4f}  elapsed={elapsed:.1f}s", flush=True)
    return running_loss/total, correct/total

def validate_product_level(model, loader, df_va, cat2idx, device):
    model.eval()
    # True label per product_id
    id2y = {int(r['_id']): cat2idx[int(r['category_id'])] for _, r in df_va.iterrows()}
    agg_logits = {}  # _id -> sum logits
    with torch.no_grad():
        for batch in loader:
            # batch: (x, _id, img_idx, y)
            x = batch[0].to(device, non_blocking=True)
            ids = batch[1].tolist()
            with autocast():
                logits = model(x).float().cpu()  # [B, C]
            for pid, logit in zip(ids, logits):
                if pid not in agg_logits:
                    agg_logits[pid] = logit.clone()
                else:
                    agg_logits[pid] += logit
    # Compute product-level accuracy
    correct = 0
    for pid, logit in agg_logits.items():
        pred = int(logit.argmax().item())
        if pid in id2y and pred == id2y[pid]:
            correct += 1
    total = len(id2y)
    acc = correct / max(1, total)
    return acc

def run_smoke_training(epochs=1, lr=1e-3, wd=0.05):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    dl_tr, dl_va, num_classes, (df_tr, df_va) = make_dataloaders_for_smoke(batch_size=128, num_workers=12)
    cat2idx, idx2cat, _ = load_mapping('category_mapping.json')
    model = timm.create_model('convnext_tiny', pretrained=True, num_classes=num_classes)
    model.to(device)
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    scaler = GradScaler()
    best_acc = -1.0
    for ep in range(1, epochs+1):
        print(f"Epoch {ep}/{epochs}")
        tr_loss, tr_acc = train_one_epoch(model, dl_tr, optimizer, scaler, device, log_every=10)
        va_acc = validate_product_level(model, dl_va, df_va, cat2idx, device)
        print(f"  Train loss={tr_loss:.4f} acc={tr_acc:.4f} | Val product-acc={va_acc:.4f}", flush=True)
        if va_acc > best_acc:
            best_acc = va_acc
            torch.save({'model': model.state_dict(), 'val_acc': va_acc}, 'model_smoke_convnext_tiny.pt')
            print(f"  Saved checkpoint with val_acc={va_acc:.4f}")
    print(f"Best val product-acc: {best_acc:.4f}")
    return best_acc

print('Smoke training utilities ready. Call run_smoke_training(epochs=1) to validate end-to-end.')

In [ ]:
# Install missing dependency and run smoke training for 1 epoch
import sys, subprocess, time, os
from pathlib import Path

def pip(*args):
    print('> pip', *args, flush=True)
    return subprocess.run([sys.executable, '-m', 'pip', *args], check=True)

# Ensure huggingface_hub is available for timm pretrained weights without touching torch stack
if not Path('installed_hfhub.flag').exists():
    pip('install', '-c', 'constraints.txt', 'huggingface_hub==0.35.1', '--upgrade-strategy', 'only-if-needed', '--no-deps')
    Path('installed_hfhub.flag').write_text('ok')
    print('huggingface_hub installed')
else:
    print('huggingface_hub already installed')

t0 = time.time()
acc = run_smoke_training(epochs=1, lr=1e-3, wd=0.05)
print(f'Smoke training done in {time.time()-t0:.1f}s, val product-acc={acc:.4f}')

In [ ]:
# Reload timm hub to pick up newly installed huggingface_hub, then run smoke training
import importlib, timm, time
import timm.models._hub as timm_hub
importlib.reload(timm_hub)
print('Reloaded timm.models._hub; has_hf_hub should be True now.')
t0 = time.time()
acc = run_smoke_training(epochs=1, lr=1e-3, wd=0.05)
print(f'Smoke training done in {time.time()-t0:.1f}s, val product-acc={acc:.4f}')

In [ ]:
# Build full indices (train/test) and full mapping
import time
t0 = time.time()
print('Starting ensure_full_indices(smoke=False)...')
ensure_full_indices(smoke=False)
print(f'Done full indexing in {time.time()-t0:.1f}s')

In [ ]:
# FORCE full reindex (overwrite smoke parquet) to get full class mapping (~5270)
from pathlib import Path
import pandas as pd, json
print('Removing existing parquets if present...')
for p in ['train_index.parquet','test_index.parquet']:
    if Path(p).exists():
        Path(p).unlink()
        print('Deleted', p)
print('Building full train index...')
build_index_parquet('train.bson', 'train_index.parquet', is_train=True, max_docs=None, log_every=200000)
print('Building full test index...')
build_index_parquet('test.bson', 'test_index.parquet', is_train=False, max_docs=None, log_every=200000)
print('Rebuilding full category mapping...')
df_tr = pd.read_parquet('train_index.parquet', columns=['category_id'])
cats = sorted(df_tr['category_id'].dropna().astype(int).unique().tolist())
cat2idx = {int(c): i for i, c in enumerate(cats)}
idx2cat = {i: int(c) for i, c in enumerate(cats)}
payload = {'cat2idx': cat2idx, 'idx2cat': idx2cat, 'num_classes': len(cats)}
Path('category_mapping.json').write_text(json.dumps(payload))
print('Full mapping saved. num_classes =', len(cats))

In [ ]:
# Build 200k balanced subset (q-cap + 1/sqrt(freq) top-up) and persist SGKF 90/10 split
import math, numpy as np, pandas as pd
from pathlib import Path
from sklearn.model_selection import StratifiedGroupKFold

train_idx_path = 'train_index.parquet'
assert Path(train_idx_path).exists(), 'train_index.parquet not found'
df = pd.read_parquet(train_idx_path, columns=['_id','category_id','offset','n_imgs'])
df = df.dropna(subset=['category_id']).copy()
df['category_id'] = df['category_id'].astype(np.int64)

# Compute class freq
freq = df['category_id'].value_counts().rename_axis('category_id').reset_index(name='freq')
n_classes = freq.shape[0]
target_n = 200_000
q = int(math.ceil(target_n / n_classes))
print('Classes:', n_classes, 'target subset size:', target_n, 'q per class:', q)

# Per-class cap sample
df_grouped = df.groupby('category_id', group_keys=False)
def take_q(g):
    n = len(g)
    k = min(q, n) if n > 0 else 0
    if k == 0:
        return g.iloc[:0]
    return g.sample(n=k, random_state=42, replace=False)
cap_df = df_grouped.apply(take_q).reset_index(drop=True)
print('After cap sample:', len(cap_df))

# Top-up if needed with p(c) ∝ 1/sqrt(f_c) from remaining products
remaining = df.merge(cap_df[['_id']], on='_id', how='left', indicator=True)
remaining = remaining[remaining['_merge'] == 'left_only'].drop(columns=['_merge'])
if len(cap_df) < target_n and len(remaining) > 0:
    freq_map = freq.set_index('category_id')['freq'].to_dict()
    w = remaining['category_id'].map(lambda c: 1.0 / math.sqrt(freq_map.get(int(c), 1)))
    w = w / w.sum()
    need = target_n - len(cap_df)
    need = min(need, len(remaining))
    top_idx = remaining.sample(n=need, weights=w, random_state=42).index
    top_df = remaining.loc[top_idx]
    sub_df = pd.concat([cap_df, top_df], axis=0, ignore_index=True)
else:
    sub_df = cap_df
sub_df = sub_df.drop_duplicates('_id')
print('Final subset size:', len(sub_df))

# Persist subset product_ids
sub_ids = sub_df['_id'].astype(np.int64).tolist()
pd.Series(sub_ids, name='_id').to_csv('subset_200k_ids.csv', index=False)
print('Saved subset_200k_ids.csv')

# SGKF 90/10 split within subset
y = sub_df['category_id'].astype(np.int64).values
g = sub_df['_id'].astype(np.int64).values
skf = StratifiedGroupKFold(n_splits=10, shuffle=True, random_state=42)
tr_idx, va_idx = next(skf.split(sub_df, y, g))
train_ids = sub_df.iloc[tr_idx]['_id'].astype(np.int64).tolist()
val_ids = sub_df.iloc[va_idx]['_id'].astype(np.int64).tolist()
pd.Series(train_ids, name='_id').to_csv('train_ids.csv', index=False)
pd.Series(val_ids, name='_id').to_csv('val_ids.csv', index=False)
print('Saved train_ids.csv:', len(train_ids), 'val_ids.csv:', len(val_ids))

# Quick coverage check
print('Covered classes in subset:', sub_df['category_id'].nunique())
print('Covered classes in train split:', sub_df.iloc[tr_idx]['category_id'].nunique())
print('Covered classes in val split:', sub_df.iloc[va_idx]['category_id'].nunique())

In [1]:
# Train ConvNeXt-Tiny@192 on 200k subset (2-3 epochs) for quick LB check; then prepare inference util
import time, math, collections, io, numpy as np, pandas as pd, torch, torch.nn as nn, torch.optim as optim
from torch import amp
from torch.utils.data import DataLoader
import timm
from timm.utils import ModelEmaV2
from timm.scheduler.cosine_lr import CosineLRScheduler

torch.backends.cudnn.benchmark = True
torch.backends.cudnn.allow_tf32 = True
torch.backends.cuda.matmul.allow_tf32 = True
torch.set_float32_matmul_precision('high')

def make_subset_loaders(batch_size=128, num_workers=16, image_size=192):
    # Load full mapping and index
    cat2idx, idx2cat, num_classes = load_mapping('category_mapping.json')
    dfi = pd.read_parquet('train_index.parquet')
    train_ids = pd.read_csv('train_ids.csv')['_id'].astype(np.int64).tolist()
    val_ids = pd.read_csv('val_ids.csv')['_id'].astype(np.int64).tolist()
    df_tr = dfi[dfi['_id'].isin(train_ids)].reset_index(drop=True)
    df_va = dfi[dfi['_id'].isin(val_ids)].reset_index(drop=True)
    ds_tr = BSONProductTrain(df_tr, 'train.bson', cat2idx, image_size=image_size)
    ds_va = BSONImageEval(df_va, 'train.bson', include_label=True, cat2idx=cat2idx, image_size=image_size)
    dl_tr = DataLoader(ds_tr, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True,
                       persistent_workers=True, prefetch_factor=8, drop_last=True)
    dl_va = DataLoader(ds_va, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True,
                       persistent_workers=True, prefetch_factor=12)
    return dl_tr, dl_va, df_tr, df_va, num_classes, cat2idx, idx2cat

def compute_class_weights(df_tr, cat2idx):
    # w_c ∝ 1/sqrt(freq_c), normalized to mean=1
    freq = df_tr['category_id'].value_counts().to_dict()
    num_classes = len(cat2idx)
    w = np.ones(num_classes, dtype=np.float32)
    for cat, idx in cat2idx.items():
        f = freq.get(int(cat), 1)
        w[idx] = 1.0 / math.sqrt(float(f))
    w = w / (w.mean() + 1e-9)
    return torch.tensor(w, dtype=torch.float32)

def train_epoch(model, loader, optimizer, scaler, device, criterion, sched=None, accum_steps=2, log_every=200, ep_offset=0, model_ema=None, ep_offset_updates=0):
    model.train()
    total, correct, loss_sum = 0, 0, 0.0
    optimizer.zero_grad(set_to_none=True)
    t0 = time.time()
    upd_idx = 0
    for it, (x, y) in enumerate(loader):
        x = x.to(device, non_blocking=True).to(memory_format=torch.channels_last)
        y = y.to(device, non_blocking=True)
        with amp.autocast('cuda'):
            logits = model(x)
            loss = criterion(logits, y) / accum_steps
        scaler.scale(loss).backward()
        if (it + 1) % accum_steps == 0:
            # grad clip
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad(set_to_none=True)
            if model_ema is not None:
                model_ema.update(model)
            # step scheduler per optimizer update (not per micro-iter)
            if sched is not None:
                upd_idx += 1
                sched.step_update(ep_offset_updates + upd_idx)
        preds = logits.detach().argmax(1)
        total += y.size(0)
        correct += (preds == y).sum().item()
        loss_sum += loss.item() * accum_steps * x.size(0)
        if (it + 1) % log_every == 0:
            print(f"  it {it+1}/{len(loader)} loss={loss_sum/total:.4f} acc={correct/total:.4f} elapsed={time.time()-t0:.1f}s", flush=True)
    return loss_sum/total, correct/total

def validate_products(model, loader, df_va, cat2idx, device):
    model.eval()
    id2y = {int(r['_id']): cat2idx[int(r['category_id'])] for _, r in df_va.iterrows()}
    agg = {}
    cnt = {}
    with torch.no_grad():
        for batch in loader:
            x = batch[0].to(device, non_blocking=True).to(memory_format=torch.channels_last)
            ids = batch[1].tolist()
            with amp.autocast('cuda'):
                logits = model(x).float().cpu()
            for pid, logit in zip(ids, logits):
                if pid not in agg:
                    agg[pid] = torch.zeros_like(logit)
                    cnt[pid] = 0
                agg[pid] += logit
                cnt[pid] += 1
    correct = 0
    for pid, logit in agg.items():
        m = logit / max(1, cnt.get(pid, 1))
        if int(pid) in id2y and int(m.argmax().item()) == id2y[int(pid)]:
            correct += 1
    return correct / max(1, len(id2y))

def train_convnext_tiny_subset(epochs=2, lr=1e-3, wd=0.05, image_size=192, batch_size=128, accum_steps=2):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    dl_tr, dl_va, df_tr, df_va, num_classes, cat2idx, idx2cat = make_subset_loaders(batch_size=batch_size, num_workers=16, image_size=image_size)
    model = timm.create_model('convnext_tiny', pretrained=True, num_classes=num_classes).to(device)
    model = model.to(memory_format=torch.channels_last)
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    # EMA
    model_ema = ModelEmaV2(model, decay=0.9997)
    # Class-weighted CE with smoothing
    cls_w = compute_class_weights(df_tr, cat2idx).to(device)
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1, weight=cls_w)
    scaler = amp.GradScaler('cuda')
    # Scheduler: 1 epoch warmup + cosine to 1e-6, step per optimizer update
    steps_per_epoch = len(dl_tr)
    updates_per_epoch = int(math.ceil(steps_per_epoch / max(1, accum_steps)))
    total_updates = updates_per_epoch * epochs
    warmup_updates = updates_per_epoch  # 1 epoch warmup
    sched = CosineLRScheduler(
        optimizer, t_initial=max(1, total_updates - warmup_updates), lr_min=1e-6,
        warmup_t=warmup_updates, warmup_lr_init=1e-5, t_in_epochs=False
    )
    best_acc = -1.0
    for ep in range(1, epochs+1):
        print(f"Epoch {ep}/{epochs}")
        ep_offset = (ep - 1) * steps_per_epoch
        ep_offset_updates = (ep - 1) * updates_per_epoch
        tr_loss, tr_acc = train_epoch(model, dl_tr, optimizer, scaler, device, criterion, sched=sched, accum_steps=accum_steps, log_every=200, ep_offset=ep_offset, model_ema=model_ema, ep_offset_updates=ep_offset_updates)
        va_acc = validate_products(model_ema.module, dl_va, df_va, cat2idx, device)
        print(f"  train loss={tr_loss:.4f} acc={tr_acc:.4f} | val product-acc={va_acc:.4f}", flush=True)
        if va_acc > best_acc:
            best_acc = va_acc
            torch.save({'model': model_ema.module.state_dict(), 'val_acc': va_acc}, 'ckpt_tiny_subset.pt')
            print(f"  Saved ckpt_tiny_subset.pt (EMA, val_acc={va_acc:.4f})")
    return best_acc

print('Subset training utilities ready. Next: run acc = train_convnext_tiny_subset(epochs=2) and then implement test inference & submission.')

Subset training utilities ready. Next: run acc = train_convnext_tiny_subset(epochs=2) and then implement test inference & submission.


In [4]:
# Launch 2-epoch training on 200k subset (ConvNeXt-Tiny@192) with EMA + cosine schedule
import time
t0 = time.time()
acc = train_convnext_tiny_subset(epochs=2, lr=1e-3, wd=0.05, image_size=192, batch_size=128, accum_steps=2)
print(f'Training finished in {time.time()-t0:.1f}s, best val product-acc={acc:.5f}', flush=True)

Epoch 1/2


  it 200/1406 loss=8.6756 acc=0.0003 elapsed=36.7s


  it 400/1406 loss=8.5689 acc=0.0018 elapsed=71.2s


  it 600/1406 loss=8.3278 acc=0.0081 elapsed=105.9s


  it 800/1406 loss=8.0837 acc=0.0171 elapsed=140.7s


...

  it 1200/1406 loss=6.2602 acc=0.1232 elapsed=211.9s


  it 1400/1406 loss=6.2481 acc=0.1248 elapsed=247.3s


  train loss=6.2467 acc=0.1249 | val product-acc=0.0573


  Saved ckpt_tiny_subset.pt (EMA, val_acc=0.0573)


Training finished in 539.5s, best val product-acc=0.05735


In [5]:
# Sanity checks: unseen val classes, label mismatches, class-weight extremes, quick val image-acc probe
import numpy as np, pandas as pd, torch

cat2idx, idx2cat, num_classes = load_mapping('category_mapping.json')
dfi = pd.read_parquet('train_index.parquet')
train_ids = pd.read_csv('train_ids.csv')['_id'].astype(np.int64).tolist()
val_ids = pd.read_csv('val_ids.csv')['_id'].astype(np.int64).tolist()
df_tr = dfi[dfi['_id'].isin(train_ids)].reset_index(drop=True)
df_va = dfi[dfi['_id'].isin(val_ids)].reset_index(drop=True)

# 1) Unseen-in-train classes check
seen_tr = set(df_tr['category_id'].dropna().astype(int).unique().tolist())
seen_va = set(df_va['category_id'].dropna().astype(int).unique().tolist())
unseen = seen_va - seen_tr
print('Unseen val classes count:', len(unseen))
print('Val products affected:', df_va[df_va['category_id'].isin(list(unseen))].shape[0])

# 2) Build val loader and check label consistency on a few batches
ds_va = BSONImageEval(df_va, 'train.bson', include_label=True, cat2idx=cat2idx, image_size=192)
from torch.utils.data import DataLoader
dl_va = DataLoader(ds_va, batch_size=128, shuffle=False, num_workers=8, pin_memory=True, persistent_workers=False, prefetch_factor=8)
id2y = {int(r['_id']): cat2idx[int(r['category_id'])] for _, r in df_va.iterrows()}
mism = 0; checked = 0
for bi, batch in enumerate(dl_va):
    _, pids, _, ys = batch
    pids = pids.tolist(); ys = ys.tolist()
    for pid, y in zip(pids, ys):
        if id2y.get(int(pid), -999) != int(y):
            mism += 1
    checked += len(pids)
    if checked >= 6400:
        break
print('Val label mismatches (first ~5k items):', mism)

# 3) Class-weight extremes
from math import sqrt
freq = df_tr['category_id'].value_counts().to_dict()
w = np.ones(len(cat2idx), dtype=np.float32)
for cat, idx in cat2idx.items():
    w[idx] = 1.0 / sqrt(float(freq.get(int(cat), 1)))
w = w / (w.mean() + 1e-9)
print('Class weight stats: min/median/max =', float(w.min()), float(np.median(w)), float(w.max()))

# 4) Quick image-level val probe on EMA checkpoint (first ~100 batches)
ckpt = torch.load('ckpt_tiny_subset.pt', map_location='cpu') if Path('ckpt_tiny_subset.pt').exists() else None
img_acc = None
if ckpt is not None:
    import timm
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = timm.create_model('convnext_tiny', pretrained=False, num_classes=num_classes).to(device).to(memory_format=torch.channels_last)
    model.load_state_dict(ckpt['model'], strict=False)
    model.eval()
    correct = 0; total = 0
    with torch.no_grad():
        for bi, batch in enumerate(dl_va):
            x = batch[0].to(device, non_blocking=True).to(memory_format=torch.channels_last)
            y = [id2y[int(pid)] for pid in batch[1].tolist()]
            y = torch.tensor(y, device=device, dtype=torch.long)
            with torch.amp.autocast('cuda') if device.type=='cuda' else torch.no_grad():
                logits = model(x)
            pred = logits.argmax(1)
            correct += (pred == y).sum().item()
            total += y.size(0)
            if bi >= 100:
                break
    img_acc = correct / max(1, total)
print('Quick image-level val acc (~100 batches):', img_acc)

Unseen val classes count: 0
Val products affected: 0


Val label mismatches (first ~5k items): 0
Class weight stats: min/median/max = 0.6879196763038635 0.958866536617279 2.5369224548339844


/tmp/ipykernel_327/1928096031.py:45: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load('ckpt_tiny_subset.pt', map_location='cpu') if Path('ckpt_tiny_subset.pt'

Quick image-level val acc (~100 batches): 0.04447710396039604


In [11]:
# Test inference with HFlip TTA, product-level aggregation, and submission.csv generation
import pandas as pd, numpy as np, torch
from pathlib import Path
import torch.multiprocessing as mp
mp.set_sharing_strategy('file_system')  # avoid /dev/shm exhaustion

def build_test_loader(image_size=192, batch_size=128, num_workers=0):
    dft = pd.read_parquet('test_index.parquet')
    ds_te = BSONImageEval(dft, 'test.bson', include_label=False, cat2idx=None, image_size=image_size)
    if num_workers <= 0:
        dl_te = DataLoader(ds_te, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=False)
    else:
        dl_te = DataLoader(ds_te, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=False,
                           persistent_workers=False, prefetch_factor=2)
    return dl_te, dft

def infer_and_submit(ckpt_path='ckpt_tiny_subset.pt', model_name='convnext_tiny', image_size=192, out_path='submission.csv'):
    assert Path(ckpt_path).exists(), f'Checkpoint not found: {ckpt_path}'
    cat2idx, idx2cat, num_classes = load_mapping('category_mapping.json')
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    import timm
    model = timm.create_model(model_name, pretrained=False, num_classes=num_classes).to(device).to(memory_format=torch.channels_last)
    ckpt = torch.load(ckpt_path, map_location='cpu')
    model.load_state_dict(ckpt['model'], strict=False)
    model.eval()
    dl_te, dft = build_test_loader(image_size=image_size, batch_size=128, num_workers=0)
    agg = {}  # pid -> summed logits (torch tensor on CPU)
    cnt = {}  # pid -> count
    with torch.no_grad():
        for i, batch in enumerate(dl_te):
            x = batch[0].to(device, non_blocking=True).to(memory_format=torch.channels_last)
            pids = batch[1].tolist()
            with amp.autocast('cuda') if device.type=='cuda' else torch.no_grad():
                logits = model(x)
                logits_fl = model(x.flip(-1))
            out = (logits + logits_fl) * 0.5
            out = out.float().cpu()
            for pid, logit in zip(pids, out):
                if pid not in agg:
                    agg[pid] = logit.clone()
                    cnt[pid] = 1
                else:
                    agg[pid] += logit
                    cnt[pid] += 1
            if (i+1) % 200 == 0:
                print(f'  infer {i+1}/{len(dl_te)} batches', flush=True)
    # Build submission following sample_submission order
    ss = pd.read_csv('sample_submission.csv')
    preds = []
    missing = 0
    for pid in ss['_id'].astype(np.int64).tolist():
        if pid in agg:
            logit = agg[pid] / max(1, cnt[pid])
            cls = int(logit.argmax().item())
            cat = int(idx2cat[cls])
        else:
            missing += 1
            cat = int(idx2cat[0])
        preds.append(cat)
    if missing:
        print('Warning: missing products in aggregation:', missing)
    sub = pd.DataFrame({'_id': ss['_id'].astype(np.int64), 'category_id': preds})
    sub.to_csv(out_path, index=False)
    print('Wrote', out_path, 'rows:', len(sub))
    return out_path

# FAST path: one image per product (first image) to speed up inference for initial LB check
class BSONProductEvalOneImage(Dataset):
    def __init__(self, index_df: pd.DataFrame, bson_path: str, image_size=192):
        self.df = index_df[['offset','_id']].reset_index(drop=True).copy()
        self.bson_path = str(bson_path)
        self._fh = None
        self.transform = T.Compose([
            T.Resize(int(image_size*1.15)),
            T.CenterCrop(image_size),
            T.ToTensor(),
            T.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
        ])
    def __len__(self):
        return len(self.df)
    def _ensure_fh(self):
        if self._fh is None:
            self._fh = open(self.bson_path, 'rb', buffering=0)
    def __getitem__(self, idx):
        r = self.df.iloc[idx]
        self._ensure_fh()
        d = read_bson_doc_at(self._fh, int(r['offset']))
        imgs = d.get('imgs', []) if d is not None else []
        pic_bytes = imgs[0]['picture'] if d is not None and imgs else None
        try:
            img = Image.open(io.BytesIO(pic_bytes)).convert('RGB') if pic_bytes is not None else Image.new('RGB',(256,256))
        except Exception:
            img = Image.new('RGB',(256,256), color=(128,128,128))
        x = self.transform(img)
        return x, int(r['_id'])

def build_test_loader_fast_products(image_size=192, batch_size=256, num_workers=0):
    dft = pd.read_parquet('test_index.parquet')
    ds = BSONProductEvalOneImage(dft, 'test.bson', image_size=image_size)
    if num_workers <= 0:
        dl = DataLoader(ds, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=False)
    else:
        dl = DataLoader(ds, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=False,
                        persistent_workers=False, prefetch_factor=2)
    return dl, dft

def infer_and_submit_fast(ckpt_path='ckpt_tiny_subset.pt', model_name='convnext_tiny', image_size=192, out_path='submission.csv'):
    assert Path(ckpt_path).exists(), f'Checkpoint not found: {ckpt_path}'
    _, idx2cat, num_classes = load_mapping('category_mapping.json')
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    import timm
    model = timm.create_model(model_name, pretrained=False, num_classes=num_classes).to(device).to(memory_format=torch.channels_last)
    ckpt = torch.load(ckpt_path, map_location='cpu')
    model.load_state_dict(ckpt['model'], strict=False)
    model.eval()
    dl, dft = build_test_loader_fast_products(image_size=image_size, batch_size=256, num_workers=0)
    preds_map = {}  # pid -> cls
    with torch.no_grad():
        for i, (x, pids) in enumerate(dl):
            x = x.to(device, non_blocking=True).to(memory_format=torch.channels_last)
            with amp.autocast('cuda') if device.type=='cuda' else torch.no_grad():
                logits = model(x)
                logits_fl = model(x.flip(-1))
            out = (logits + logits_fl) * 0.5
            cls = out.argmax(1).detach().cpu().tolist()
            for pid, c in zip(pids.tolist(), cls):
                preds_map[int(pid)] = int(c)
            if (i+1) % 200 == 0:
                print(f'  fast infer {i+1}/{len(dl)} batches', flush=True)
    ss = pd.read_csv('sample_submission.csv')
    preds = []
    missing = 0
    for pid in ss['_id'].astype(np.int64).tolist():
        if int(pid) in preds_map:
            cat = int(idx2cat[preds_map[int(pid)]])
        else:
            missing += 1
            cat = int(idx2cat[0])
        preds.append(cat)
    if missing:
        print('Warning: missing products in fast inference:', missing)
    sub = pd.DataFrame({'_id': ss['_id'].astype(np.int64), 'category_id': preds})
    sub.to_csv(out_path, index=False)
    print('Wrote', out_path, 'rows:', len(sub))
    return out_path

print('Inference utilities ready. Call infer_and_submit(...) for full per-image agg, or infer_and_submit_fast(...) for one-image-per-product baseline.')

Inference utilities ready. Call infer_and_submit(...) for full per-image agg, or infer_and_submit_fast(...) for one-image-per-product baseline.


In [12]:
# Generate submission quickly using fast one-image-per-product inference (HFlip TTA)
out_path = infer_and_submit_fast(ckpt_path='ckpt_tiny_subset.pt', model_name='convnext_tiny', image_size=192, out_path='submission.csv')
import pandas as pd
sub = pd.read_csv(out_path)
print('submission shape:', sub.shape)
print(sub.head())

/tmp/ipykernel_327/2226608465.py:113: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ckpt_path, map_location='cpu')


  fast infer 200/2762 batches


  fast infer 400/2762 batches


  fast infer 600/2762 batches


  fast infer 800/2762 batches


...

  fast infer 2000/2762 batches


  fast infer 2200/2762 batches


  fast infer 2400/2762 batches


  fast infer 2600/2762 batches


Wrote submission.csv rows: 706990
submission shape: (706990, 2)
   _id  category_id
0    6   1000012546
1    7   1000012546
2   12   1000001844
3   59   1000019193
4   61   1000010633


In [13]:
# ConvNeXt-Base@224 training on 200k subset with EMA, cosine, early stopping
import math, time, numpy as np, pandas as pd, torch, torch.nn as nn, torch.optim as optim
from torch import amp
from torch.utils.data import DataLoader
import timm
from timm.utils import ModelEmaV2
from timm.scheduler.cosine_lr import CosineLRScheduler

def make_subset_loaders_base(batch_size=64, num_workers=16, image_size=224):
    cat2idx, idx2cat, num_classes = load_mapping('category_mapping.json')
    dfi = pd.read_parquet('train_index.parquet')
    train_ids = pd.read_csv('train_ids.csv')['_id'].astype(np.int64).tolist()
    val_ids = pd.read_csv('val_ids.csv')['_id'].astype(np.int64).tolist()
    df_tr = dfi[dfi['_id'].isin(train_ids)].reset_index(drop=True)
    df_va = dfi[dfi['_id'].isin(val_ids)].reset_index(drop=True)
    ds_tr = BSONProductTrain(df_tr, 'train.bson', cat2idx, image_size=image_size)
    ds_va = BSONImageEval(df_va, 'train.bson', include_label=True, cat2idx=cat2idx, image_size=image_size)
    dl_tr = DataLoader(ds_tr, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True,
                       persistent_workers=True, prefetch_factor=8, drop_last=True)
    dl_va = DataLoader(ds_va, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True,
                       persistent_workers=True, prefetch_factor=12)
    return dl_tr, dl_va, df_tr, df_va, num_classes, cat2idx, idx2cat

def compute_class_weights_from_df(df_tr, cat2idx):
    import numpy as np, math as _m
    freq = df_tr['category_id'].value_counts().to_dict()
    w = np.ones(len(cat2idx), dtype=np.float32)
    for cat, idx in cat2idx.items():
        f = float(freq.get(int(cat), 1))
        w[idx] = 1.0 / _m.sqrt(max(f, 1.0))
    w = w / (w.mean() + 1e-9)
    return torch.tensor(w, dtype=torch.float32)

def validate_products_base(model, loader, df_va, cat2idx, device):
    model.eval()
    id2y = {int(r['_id']): cat2idx[int(r['category_id'])] for _, r in df_va.iterrows()}
    agg, cnt = {}, {}
    with torch.no_grad():
        for batch in loader:
            x = batch[0].to(device, non_blocking=True).to(memory_format=torch.channels_last)
            ids = batch[1].tolist()
            with amp.autocast('cuda') if device.type == 'cuda' else torch.no_grad():
                logits = model(x).float().cpu()
            for pid, logit in zip(ids, logits):
                if pid not in agg:
                    agg[pid] = logit.clone(); cnt[pid] = 1
                else:
                    agg[pid] += logit; cnt[pid] += 1
    correct = 0
    for pid, logit in agg.items():
        m = logit / max(1, cnt.get(pid, 1))
        if int(pid) in id2y and int(m.argmax().item()) == id2y[int(pid)]:
            correct += 1
    total = len(id2y)
    return correct / max(1, total)

def train_convnext_base_subset(epochs=10, lr=8e-4, wd=0.05, image_size=224, batch_size=64, accum_steps=4, patience=2, drop_path=0.2, ckpt_path='ckpt_convnextb_subset.pt'):
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.allow_tf32 = True
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.set_float32_matmul_precision('high')
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    dl_tr, dl_va, df_tr, df_va, num_classes, cat2idx, idx2cat = make_subset_loaders_base(batch_size=batch_size, num_workers=16, image_size=image_size)
    model = timm.create_model('convnext_base', pretrained=True, num_classes=num_classes, drop_path_rate=drop_path).to(device).to(memory_format=torch.channels_last)
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=wd, betas=(0.9,0.999))
    model_ema = ModelEmaV2(model, decay=0.9997)
    cls_w = compute_class_weights_from_df(df_tr, cat2idx).to(device)
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1, weight=cls_w)
    scaler = amp.GradScaler('cuda') if device.type == 'cuda' else None
    steps_per_epoch = len(dl_tr)
    updates_per_epoch = int(math.ceil(steps_per_epoch / max(1, accum_steps)))
    total_updates = updates_per_epoch * epochs
    warmup_updates = updates_per_epoch  # 1 epoch warmup
    sched = CosineLRScheduler(
        optimizer, t_initial=max(1, total_updates - warmup_updates), lr_min=1e-6,
        warmup_t=warmup_updates, warmup_lr_init=1e-5, t_in_epochs=False
    )
    best_acc = -1.0
    no_improve = 0
    print(f'Start training: epochs={epochs}, eff_batch~={batch_size*accum_steps}, lr={lr}, wd={wd}, img={image_size}', flush=True)
    for ep in range(1, epochs + 1):
        t0 = time.time()
        model.train()
        total, correct, loss_sum = 0, 0, 0.0
        optimizer.zero_grad(set_to_none=True)
        upd_idx = 0
        for it, (x, y) in enumerate(dl_tr):
            x = x.to(device, non_blocking=True).to(memory_format=torch.channels_last)
            y = y.to(device, non_blocking=True)
            if scaler is not None:
                with amp.autocast('cuda'):
                    logits = model(x)
                    loss = criterion(logits, y) / accum_steps
                scaler.scale(loss).backward()
            else:
                logits = model(x)
                loss = criterion(logits, y) / accum_steps
                loss.backward()
            if (it + 1) % accum_steps == 0:
                if scaler is not None:
                    scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                if scaler is not None:
                    scaler.step(optimizer); scaler.update()
                else:
                    optimizer.step()
                optimizer.zero_grad(set_to_none=True)
                model_ema.update(model)
                upd_idx += 1
                sched.step_update((ep - 1) * updates_per_epoch + upd_idx)
            preds = logits.detach().argmax(1)
            total += y.size(0)
            correct += (preds == y).sum().item()
            loss_sum += loss.item() * accum_steps * x.size(0)
            if (it + 1) % 100 == 0:
                print(f'E{ep} it {it+1}/{len(dl_tr)} loss={loss_sum/max(1,total):.4f} acc={correct/max(1,total):.4f} elapsed={time.time()-t0:.1f}s', flush=True)
        tr_loss = loss_sum / max(1, total)
        tr_acc = correct / max(1, total)
        va_acc = validate_products_base(model_ema.module, dl_va, df_va, cat2idx, device)
        print(f'Epoch {ep}: train loss={tr_loss:.4f} acc={tr_acc:.4f} | val product-acc={va_acc:.4f} | took {time.time()-t0:.1f}s', flush=True)
        if va_acc > best_acc:
            best_acc = va_acc; no_improve = 0
            torch.save({'model': model_ema.module.state_dict(), 'val_acc': va_acc}, ckpt_path)
            print(f'  Saved {ckpt_path} (EMA) with val_acc={va_acc:.4f}', flush=True)
        else:
            no_improve += 1
            if no_improve >= patience:
                print(f'Early stopping at epoch {ep} (no improvement {no_improve}/{patience})', flush=True)
                break
    print('Best val product-acc:', f'{best_acc:.4f}')
    return best_acc

print('ConvNeXt-Base training function ready. Next: execute train_convnext_base_subset(...) to launch the strong run.')

ConvNeXt-Base training function ready. Next: execute train_convnext_base_subset(...) to launch the strong run.


In [14]:
# Launch ConvNeXt-Base@224 strong run on 200k subset
import os, time, torch
os.environ.setdefault('OMP_NUM_THREADS', '1')
os.environ.setdefault('MKL_NUM_THREADS', '1')
print('CUDA available:', torch.cuda.is_available())
if torch.cuda.is_available():
    print('GPU:', torch.cuda.get_device_name(0))
t0 = time.time()
best_acc = train_convnext_base_subset(
    epochs=10, lr=8e-4, wd=0.05, image_size=224,
    batch_size=64, accum_steps=4, patience=2, drop_path=0.2,
    ckpt_path='ckpt_convnextb_subset.pt'
)
print(f'Total training time: {time.time()-t0:.1f}s | best val product-acc={best_acc:.4f}', flush=True)

CUDA available: True
GPU: NVIDIA A10-24Q


Start training: epochs=10, eff_batch~=256, lr=0.0008, wd=0.05, img=224


E1 it 100/2812 loss=8.7830 acc=0.0006 elapsed=35.8s


E1 it 200/2812 loss=8.7411 acc=0.0004 elapsed=63.5s


E1 it 300/2812 loss=8.6976 acc=0.0005 elapsed=91.2s


...

E10 it 2800/2812 loss=1.7883 acc=0.9160 elapsed=791.1s


Epoch 10: train loss=1.7882 acc=0.9160 | val product-acc=0.4663 | took 849.0s


  Saved ckpt_convnextb_subset.pt (EMA) with val_acc=0.4663


Best val product-acc: 0.4663


Total training time: 8509.6s | best val product-acc=0.4663


In [15]:
# Inference with ConvNeXt-Base EMA checkpoint: full per-image aggregation + HFlip TTA
from pathlib import Path
import pandas as pd
ckpt_path = 'ckpt_convnextb_subset.pt'
assert Path(ckpt_path).exists(), 'ConvNeXt-Base checkpoint not found. Train run must complete first.'
out_path = infer_and_submit(ckpt_path=ckpt_path, model_name='convnext_base', image_size=224, out_path='submission_base.csv')
sub = pd.read_csv(out_path)
print('submission_base shape:', sub.shape)
print(sub.head())

/tmp/ipykernel_327/2226608465.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ckpt_path, map_location='cpu')


  infer 200/9671 batches


  infer 400/9671 batches


  infer 600/9671 batches


  infer 800/9671 batches


...

  infer 9200/9671 batches


  infer 9400/9671 batches


  infer 9600/9671 batches


Wrote submission_base.csv rows: 706990


submission_base shape: (706990, 2)
   _id  category_id
0    6   1000005744
1    7   1000004079
2   12   1000018286
3   59   1000007138
4   61   1000010653


In [16]:
# Copy best submission to official filename and quick sanity check
import pandas as pd, shutil, os
assert os.path.exists('submission_base.csv'), 'submission_base.csv not found'
shutil.copyfile('submission_base.csv', 'submission.csv')
sub = pd.read_csv('submission.csv')
print('submission.csv shape:', sub.shape)
print(sub.head())

submission.csv shape: (706990, 2)
   _id  category_id
0    6   1000005744
1    7   1000004079
2   12   1000018286
3   59   1000007138
4   61   1000010653


In [ ]:
# Multi-scale TTA inference: add Resize 240 -> CenterCrop 224 alongside 256->224 + HFlip
import pandas as pd, numpy as np, torch
from pathlib import Path
from torch.utils.data import DataLoader
import torchvision.transforms as T

class BSONImageEvalScaled(Dataset):
    def __init__(self, index_df: pd.DataFrame, bson_path: str, include_label: bool, cat2idx: dict | None, resize_size: int, crop_size: int):
        rows = []
        for _, r in index_df.iterrows():
            for k in range(int(r['n_imgs'])):
                rows.append((int(r['offset']), int(r['_id']), k, int(r['category_id']) if include_label and pd.notna(r['category_id']) else None))
        self.df = pd.DataFrame(rows, columns=['offset','_id','img_idx','category_id'])
        self.df.sort_values('offset', inplace=True)
        self.bson_path = str(bson_path)
        self.include_label = include_label
        self.cat2idx = cat2idx
        self._fh = None
        self.transform = T.Compose([
            T.Resize(resize_size),
            T.CenterCrop(crop_size),
            T.ToTensor(),
            T.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
        ])
    def __len__(self):
        return len(self.df)
    def _ensure_fh(self):
        if self._fh is None:
            self._fh = open(self.bson_path, 'rb', buffering=0)
    def __getitem__(self, idx):
        r = self.df.iloc[idx]
        self._ensure_fh()
        d = read_bson_doc_at(self._fh, int(r['offset']))
        imgs = d.get('imgs', []) if d is not None else []
        pic_bytes = imgs[int(r['img_idx'])]['picture'] if d is not None and imgs else None
        try:
            img = Image.open(io.BytesIO(pic_bytes)).convert('RGB') if pic_bytes is not None else Image.new('RGB',(256,256))
        except Exception:
            img = Image.new('RGB',(256,256), color=(128,128,128))
        x = self.transform(img)
        if self.include_label:
            cat_id = int(r['category_id']) if r['category_id'] is not None and pd.notna(r['category_id']) else None
            y = self.cat2idx.get(cat_id, 0) if (self.cat2idx is not None and cat_id is not None) else -1
            return x, int(r['_id']), int(r['img_idx']), y
        else:
            return x, int(r['_id']), int(r['img_idx'])

def infer_and_submit_multiscale(ckpt_path: str, model_name: str, crop_size=224, scales=(256, 240), out_path='submission_ms.csv'):
    assert Path(ckpt_path).exists(), f'Checkpoint not found: {ckpt_path}'
    cat2idx, idx2cat, num_classes = load_mapping('category_mapping.json')
    import timm
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = timm.create_model(model_name, pretrained=False, num_classes=num_classes).to(device).to(memory_format=torch.channels_last)
    ckpt = torch.load(ckpt_path, map_location='cpu')
    model.load_state_dict(ckpt['model'], strict=False)
    model.eval()
    dft = pd.read_parquet('test_index.parquet')
    agg, cnt = {}, {}  # pid -> logits sum, count
    with torch.no_grad():
        for s in scales:
            ds = BSONImageEvalScaled(dft, 'test.bson', include_label=False, cat2idx=None, resize_size=s, crop_size=crop_size)
            dl = DataLoader(ds, batch_size=128, shuffle=False, num_workers=0, pin_memory=False)
            for i, batch in enumerate(dl):
                x = batch[0].to(device, non_blocking=True).to(memory_format=torch.channels_last)
                pids = batch[1].tolist()
                with amp.autocast('cuda') if device.type=='cuda' else torch.no_grad():
                    logits = model(x)
                    logits_fl = model(x.flip(-1))
                out = (logits + logits_fl) * 0.5
                out = out.float().cpu()
                for pid, logit in zip(pids, out):
                    if pid not in agg:
                        agg[pid] = logit.clone(); cnt[pid] = 1
                    else:
                        agg[pid] += logit; cnt[pid] += 1
                if (i+1) % 200 == 0:
                    print(f'scale {s}: infer {i+1}/{len(dl)} batches', flush=True)
    ss = pd.read_csv('sample_submission.csv')
    preds = []
    missing = 0
    for pid in ss['_id'].astype(np.int64).tolist():
        if pid in agg:
            logit = agg[pid] / max(1, cnt[pid])
            cls = int(logit.argmax().item())
            cat = int(idx2cat[cls])
        else:
            missing += 1
            cat = int(idx2cat[0])
        preds.append(cat)
    if missing:
        print('Warning: missing products in aggregation:', missing)
    sub = pd.DataFrame({'_id': ss['_id'].astype(np.int64), 'category_id': preds})
    sub.to_csv(out_path, index=False)
    print('Wrote', out_path, 'rows:', len(sub))
    return out_path

# Run multi-scale TTA on ConvNeXt-Base EMA checkpoint and prepare submission.csv
ms_out = infer_and_submit_multiscale(ckpt_path='ckpt_convnextb_subset.pt', model_name='convnext_base', crop_size=224, scales=(256,240), out_path='submission_ms.csv')
import shutil
shutil.copyfile(ms_out, 'submission.csv')
print('submission.csv updated from multi-scale TTA:', ms_out)

/tmp/ipykernel_327/2072974973.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ckpt_path, map_location='cpu')
